In [12]:
import psycopg2
from psycopg2 import sql
import sqlite3
import json

# Database connection settings
db_settings = {
    'dbname': 'wikiknowledge',
    'user': 'ocelot',
    'password': 'odinson',
    'host': 'localhost'
}

sqlite_db_path = 'database/final_embedding.db'  # Path to your SQLite database
link_embeddings_path = 'wiki_knowledge/embeddings/link_embeddings.json'  # Path to your link_embeddings.json file

# Load link embeddings from JSON file
with open(link_embeddings_path, 'r') as f:
    link_embeddings = json.load(f)

def get_vector_by_document_pg(conn, document_name: str):
    """Retrieve the vector from the PostgreSQL database using the document name."""
    with conn.cursor() as cur:
        try:
            cur.execute(sql.SQL("SELECT embedding FROM my_vectors WHERE document = %s"), [document_name])
            result = cur.fetchone()
            if result:
                return result[0]
            else:
                print(f"No vector found for document: {document_name}")
                conn.rollback()  # Rollback if no document is found
                return None
        except Exception as e:
            print(f"Error querying for document '{document_name}': {e}")
            conn.rollback()
            return None

def get_vector_by_document_sqlite(document_name: str):
    """Retrieve the vector from the SQLite database using the document name and link_embeddings.json."""
    try:
        emb_idx = link_embeddings.get(document_name)
        if emb_idx is None:
            print(f"No embedding index found for document: {document_name}")
            return None

        conn = sqlite3.connect(sqlite_db_path)
        cursor = conn.cursor()
        cursor.execute('SELECT "values" FROM embeddings WHERE emb_idx = ?', (emb_idx,))
        row = cursor.fetchone()
        conn.close()

        if row:
            return json.loads(row[0])  # Assuming the vector is stored as a JSON array
        else:
            print(f"No vector found for embedding index: {emb_idx}")
            return None
    except Exception as e:
        print(f"Error querying SQLite database for document '{document_name}': {e}")
        return None

def get_vector_by_document(conn, document_name: str):
    """Retrieve the vector from the PostgreSQL database, fallback to SQLite database if not found."""
    vector = get_vector_by_document_pg(conn, document_name)
    if vector is None:
        vector = get_vector_by_document_sqlite(document_name)
    return vector

def query_db(conn, query_vector):
    """Query the PostgreSQL database for the nearest neighbors using the given vector."""
    with conn.cursor() as cur:
        try:
            cur.execute(sql.SQL("""
                SELECT document, (embedding <-> %s) AS distance
                FROM my_vectors
                ORDER BY distance ASC
                LIMIT 20
            """), [query_vector])
            results = cur.fetchall()
            return results
        except Exception as e:
            print(f"Error querying database: {e}")
            conn.rollback()
            return []

# Connect to PostgreSQL
conn = psycopg2.connect(**db_settings)


In [14]:
# Document name to search
document_name = "Donald trump"

# Get the vector from the database
query_vector = get_vector_by_document(conn, document_name)
print("We have the query vector")

if query_vector:
    # Query the database using the vector
    results = query_db(conn, query_vector)
    
    print(f"Query results for document '{document_name}':")
    for document, distance in results:
        print(f"Document: {document}, Distance: {distance}")

No vector found for document: Donald trump
We have the query vector


In [ ]:
# Close the connection
conn.close()